In [1]:
import numpy as np
import pandas as pd
import sklearn as sl

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
def Split_dataset(file,percent):
    name=[i for i in range(len(file[0]))]
    test_data=pd.DataFrame(columns=name)
    train_data=pd.DataFrame(columns=name)
    df = pd.DataFrame(file)
    print(len(df))
    for i in range(len(df)):                   
        if np.random.rand()<percent:
            test_data.loc[len(test_data)]=list(df.loc[i])
        else:
            train_data.loc[len(train_data)]=list(df.loc[i])
        if i%300==0:
            print(round((i/len(df))*100),'%',end="|")
    print('100 %')

    return test_data,train_data

In [4]:
def Similarity(data_instances1,data_instances2,l=4):#Euclidean metric
    return np.sqrt(sum([(data_instances1[i]-data_instances2[i])**2 for i in range(l)]))

In [5]:
def get_kNeighbors(train_data,data_instances,k,radius=False):
    Neighbors = [[Similarity(data_instances,train_data.loc[i].tolist()),train_data.loc[i].tolist()]for i in range(len(train_data))]
    Neighbors=sorted(Neighbors,key=lambda x:x[0])
    
    if radius:
        l=0
        for i in Neighbors:
            #print('%%',i[0],' ',k,'%%')
            if i[0]<k:
                l+=1
            else:
                if l==0:
                    l=1
                break
        return Neighbors[:l]
    else:
        return Neighbors[:k]

In [6]:
def get_Response(Neighbors):
    Iris_type = {}
    for j in Neighbors:
        if j[1][-1] not in Iris_type:
            Iris_type[j[1][-1]]=1
        else:
            Iris_type[j[1][-1]]+=1
    prediction=[0,'']
    for j in Iris_type.keys():
        if Iris_type[j]>prediction[0]:
            prediction[0]=Iris_type[j]
            prediction[1]=j
    return prediction[-1]

In [7]:
def get_accuracy(train_data,test_data,k_Neighbors,radius=False):
    success=0
    for i in range(len(test_data)):
        test=test_data.loc[i].tolist()
        print('\n\nПроверка: ',test[-5:])
        Neighbors = get_kNeighbors(train_data,test,k_Neighbors,radius=radius)
        print('\nБлижайшние данные к проверяемым:\n',[i[1][-1] for i in Neighbors])
        prediction = get_Response(Neighbors)
        print('Предсказание:\n',prediction)
        if test[-1]==prediction:
            success+=1
            print('\nПредсказание верное\n\n\n')
        print("Выполнено:",i/len(test_data),'%', 'Успешно:',success)
    return (success/len(test_data))

In [8]:
def get_data(path,files):
    C_10=[0,0]
    for i in range(len(files)):
        print('Loading:',path+files[i])
        tmp_file=unpickle(path+files[i])  
        if i==0:
            C_10[1]=tmp_file[b'labels']
            C_10[0]=tmp_file[b'data'].tolist()
        else:
            C_10[1]+=tmp_file[b'labels']
            C_10[0]+=tmp_file[b'data'].tolist()
#         print(len(C_10[0])) 
#         print(len(C_10[1]))  
#         print(len(C_10),'\n')
    
    for i in range(len(C_10[0])):
        C_10[0][i]+=[C_10[1][i]]
        if i%10000==0:
            print(round((i/len(C_10[0]))*100),'%')
    print('100 %')
    return C_10[0]

In [ ]:
path='cifar-10-python/cifar-10-batches-py/'
files_train=['data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5']#
files_test=['test_batch']
C_10_train=get_data(path,files_train)
C_10_test=get_data(path,files_test)

Loading: cifar-10-python/cifar-10-batches-py/data_batch_1
Loading: cifar-10-python/cifar-10-batches-py/data_batch_2
Loading: cifar-10-python/cifar-10-batches-py/data_batch_3
Loading: cifar-10-python/cifar-10-batches-py/data_batch_4
Loading: cifar-10-python/cifar-10-batches-py/data_batch_5
0 %
20 %
40 %
60 %
80 %
100 %
Loading: cifar-10-python/cifar-10-batches-py/test_batch
0 %
100 %


In [ ]:
print(len(C_10_train[0]))

3073


In [ ]:
# border = 10000
# test_data_size=2000
# percent=test_data_size/float(border)
test_data=pd.DataFrame(C_10_test[:2500])
train_data=pd.DataFrame(C_10_train)
# test_data,train_data=Split_dataset(C_10_test,1)
# test_data,train_data=Split_dataset(C_10_train,0)

In [ ]:
print(len(test_data))#
print(len(train_data))#
print(len(train_data)+len(test_data))#

2500
50000
52500


In [ ]:
k_Neighbors=20
rk_Neighbors=15
get_accuracy(train_data,test_data,rk_Neighbors,radius=True)



Проверка:  [145, 124, 129, 110, 3]

Ближайшние данные к проверяемым:
 [6, 7, 0, 9, 8, 2, 8, 2, 9, 0, 8, 7, 1, 4, 0, 9, 8, 2, 2, 6, 2, 5, 8, 8, 5, 0, 0, 7, 5, 9, 3, 4, 7, 0, 9, 9, 3, 0, 9, 8, 0, 2, 0, 7, 3, 5, 3, 0, 8, 9, 0, 9, 2, 8, 4, 8, 9, 6, 3, 4, 9, 3, 0, 2, 3, 8, 8, 0, 1, 8, 8, 8, 3, 9, 2, 0, 2, 1, 3, 2, 8, 8, 4, 2, 8, 8, 7, 3, 9, 3, 0, 4, 3, 0, 0, 4, 4, 8, 5, 9, 8, 0, 8, 6, 5, 0, 5, 3, 7, 2, 3, 9, 0, 3, 0, 5, 1, 8, 8, 4, 2, 4, 3, 9, 7, 8, 3, 2, 1, 2, 0, 8, 0, 2, 0, 4, 1, 3, 5, 5, 5, 0, 8, 4, 4, 7, 7, 2, 1, 9, 0, 9, 5, 1, 8, 6, 8, 3, 4, 4, 0, 5, 9, 3, 9, 4, 3, 8, 7, 1, 6, 1, 4, 5, 0, 0, 0, 3, 4, 2, 6, 0, 8, 2, 8, 9, 2, 0, 2, 0, 0, 4, 8, 1, 0, 5, 1, 3, 7, 7, 3, 0, 6, 5, 0, 7, 2, 0, 9, 7, 2, 4, 4, 2, 8, 0, 3, 0, 5, 8, 5, 4, 0, 3, 7, 8, 9, 4, 5, 1, 3, 2, 2, 0, 2, 8, 8, 5, 4, 8, 7, 5, 8, 5, 3, 8, 7, 2, 5, 0, 0, 4, 0, 8, 2, 1, 9, 4, 1, 5, 4, 0, 7, 4, 2, 8, 8, 7, 2, 9, 9, 2, 1, 3, 0, 7, 2, 8, 8, 3, 2, 7, 2, 1, 7, 8, 8, 0, 9, 0, 2, 2, 7, 4, 2, 2, 7, 0, 2, 7, 8, 9, 0, 0, 8, 3, 5, 8, 8, 